<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 2<br/>*POS taggers* pour le français dans spaCy et NLTK

**Objectif**

Comparer l'étiqueteur morphosyntaxique français prêt-à-l'emploi de spaCy avec deux étiqueteurs entraînés, l'un dans spaCy et l'autre dans NLTK.

## 1. Installation et test de spaCy

La boîte à outils spaCy est une librairie Python *open source* pour le TAL, dédiée à un usage en production. Les documents suivants vous seront utiles :
* comment [installer](https://spacy.io/usage) spaCy
* comment [télécharger un modèle](https://spacy.io/usage/models) pour une langue donnée (on appelle ces modèles des *trained pipelines* car ils enchaînent plusieurs traitements)
* comment faire les [premiers pas](https://spacy.io/usage/spacy-101) dans l'utilisation de spaCy

Veuillez installer spaCy, puis la *pipeline* pour le français appelée `fr_core_news_sm`.  Si vous utilisez *conda*, installez spaCy dans l'environnement du cours TAL.

In [101]:
import spacy
import tqdm # permet l'affichage d'une barre de progression

**1a.** Une pipeline effectue un ensemble de traitements d'un texte en lui ajoutant des annotations.  Les traitements effectués par la pipeline `fr_core_news_sm` sont [documentés ici](https://spacy.io/models/fr#fr_core_news_sm).  La liste des traitements d'une pipeline figure dans son attribut `.pipe_names`.  On peut activer ou désactiver un traitement T avec, respectivement, les méthodes `.disable_pipe(T)` et `.enable_pipe(T)` appliquées à la pipeline.

* Veuillez afficher les traitements disponibles dans la pipeline `fr_core_news_sm` chargée ci-dessus sous le nom de `nlp` .
* Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
* Vérifiez que la désactivation a bien fonctionné en affichant les traitements activés.

In [ ]:
# Veuillez écrire votre code ici.
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

traitements = nlp.pipe_names
print("Tous les traitements : ")
for t in traitements:
    print(t)
    match t:
        case "tok2vec":
            pass  
        case "morphologizer":
            pass
        case _:
            nlp.disable_pipe(t)

traitements = nlp.pipe_names
print("Tous les traitements activés :")
for t in traitements:
    print(t)


Tous les traitements : 
tok2vec
morphologizer
parser
attribute_ruler
lemmatizer
ner
Tous les traitements activés :
tok2vec
morphologizer


In [103]:
from spacy.lang.fr.examples import sentences

**1b.** L'objet `sentences` chargé ci-dessus contient une liste de phrases en français. 

* Veuillez afficher les deux premières phrases de `sentences`.
* Veuillez analyser chacune de ces deux phrases avec la pipeline `nlp` puis afficher chaque token et son POS tag.
    * indication : aidez-vous de la [documentation](https://spacy.io/models/fr#fr_core_news_sm) de `fr_core_news_sm`
    * consigne d'affichage : indiquer le tag entre crochets après chaque token, comme ceci : Les \[DET\] robots \[NOUN\] ...
    * note : la documentation détaillée du POS tagging dans spaCy est [disponible ici](https://spacy.io/usage/linguistic-features)
* Veuillez commenter la tokenisation et les POS tags observés : vous semblent-ils corrects pour les deux phrases ?

In [104]:
# Veuillez écrire votre code et votre commentaire ici.
# Afficher les deux premières phrases de `sentences`

doc1 = nlp(sentences[0])
print(doc1)
doc2 = nlp(sentences[1])
print(doc2)

docs = [doc1,doc2]

for doc in docs :
    for token in doc:
        print(f"{token.text} [{token.pos_}]")
        
#cherche est considéré comme nom, start-up devrait être un mot simple, donc up n'est pas un adjectif.
#Il y a encore anglaise qui devrait être un adjectif pour start-up. Et finalement déplacent n'est pas un adverbe mais un verbe conjugué
#Sinon je trouve pas d'autres erreurs, on obtient alors un pourcentage d'accuracy de 20/26 donc pas super... (à vérifier)





Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars
Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs
Apple [NOUN]
cherche [NOUN]
à [ADP]
acheter [VERB]
une [DET]
start [NOUN]
- [NOUN]
up [ADJ]
anglaise [NOUN]
pour [ADP]
1 [NUM]
milliard [NOUN]
de [ADP]
dollars [NOUN]
Les [DET]
voitures [NOUN]
autonomes [ADJ]
déplacent [ADV]
la [DET]
responsabilité [NOUN]
de [ADP]
l' [DET]
assurance [NOUN]
vers [ADP]
les [DET]
constructeurs [NOUN]


## 2. Prise en main des données

Les données sont fournies dans un format tabulaire dans l'archive `UD_French-GSD.zip` sur Cyberlearn.  Elles sont basées sur les données fournies par le projet [Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD).  Leur format, appelé CoNLL-U, est [documenté ici](https://universaldependencies.org/format.html).  Veuillez placer les trois fichiers contenus dans l'archive dans un sous-dossier de ce notebook nommé `spacy_data`.

Les trois fichiers contiennent des phrases en français annotées avec les POS tags :
* le fichier `fr-ud-train.conllu` est destiné à l'entraînement
* le fichier `fr-ud-dev.conllu` est destiné aux tests préliminaires et aux réglages des paramètres
* le fichier `fr-ud-test.conllu` est destiné à l'évaluation finale.

**2a.** En inspectant les fichiers avec un éditeur texte, veuillez déterminer dans quelle colonne se trouvent les *tokens* des textes originaux, et dans quelle colonne se trouvent leurs étiquettes morpho-syntaxiques correctes (*POS tags*).  Que contient la troisième colonne ?

In [105]:
# Veuillez écrire vos réponses dans cette cellule.
# Les tokens se trouvent dans la colonne numéro 2
# Les POS tags sont dans la colonne 4 et la colonne 3 indique le token après lemming

**2b.** Veuillez convertir les trois fichiers de données en des fichiers binaires utilisables par spaCy, en utilisant la [commande 'convert' fournie par spaCy](https://spacy.io/api/cli#convert).  La commande est donnée ci-dessous, le premier dossier `./input_data` contient les 3 fichiers `.conllu` et le dossier `./spacy-data` contiendra les 3 résultats.

* Veuillez exécuter la commande de conversion.
* Combien de phrases environ (à 10 phrases près) contient chaque fichier (*train*, *dev*, *test*) ?  Observez la commande et son résultat pour répondre.

In [106]:
!python -m spacy convert ./input_data ./spacy_data --converter conllu  --n-sents 10 --lang fr

ℹ Grouping every 10 sentences into a document.
✔ Generated output file (148 documents): spacy_data\fr-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (42 documents): spacy_data\fr-ud-test.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1456 documents):
spacy_data\fr-ud-train.spacy


In [107]:
# Veuillez indiquer les nombres de phrases ici.
# Je pense qu'il y a 1480 phrases pour le dev
# 420 phrases pour le test et 14560 phrases pour le train

**2c**. Les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Dans notre cas, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la première phrase des données de test dans une variable nommée `premiere_phrase_test`.
* Veuillez afficher cette phrase, ainsi que son type dans spaCy.

In [108]:
from spacy.tokens import DocBin
from spacy.tokens import Doc
test_data = DocBin().from_disk("./spacy_data/fr-ud-test.spacy")
# Exemple d'utilisation (afficher toutes les phrases)
# for doc in test_data.get_docs(nlp.vocab): 
#     for sent in doc.sents:
#         print(sent)

In [109]:
# Veuillez écrire votre code ici.
premiere_phrase_test = next(next(test_data.get_docs(nlp.vocab)).sents)

print(premiere_phrase_test)
print(type(premiere_phrase_test))


Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.
<class 'spacy.tokens.span.Span'>


## 3. Évaluation du POS tagger français de la pipeline `fr_core_news_sm`

**3a.** Veuillez effectuer le *POS tagging* avec spaCy de la `premiere_phrase_test` et afficher les résultats dans le format demandé au (1b).  Indication : convertissez la `premiere_phrase_test` dans un objet de type `Doc` en lui appliquant la méthode `.as_doc()`.  Cet objet peut être ensuite traité par la pipeline `nlp`.

In [110]:
# Veuillez écrire votre code ici.
premiere_phrase_test_doc = nlp(premiere_phrase_test.as_doc())
for token in premiere_phrase_test_doc:
        print(f"{token.text} [{token.pos_}]")


Je [PRON]
sens [VERB]
qu' [SCONJ]
entre [ADP]
ça [PRON]
et [CCONJ]
les [DET]
films [NOUN]
de [ADP]
médecins [NOUN]
et [CCONJ]
scientifiques [NOUN]
fous [PRON]
que [PRON]
nous [PRON]
avons [AUX]
déjà [ADV]
vus [VERB]
, [PUNCT]
nous [PRON]
pourrions [VERB]
emprunter [VERB]
un [DET]
autre [ADJ]
chemin [NOUN]
pour [ADP]
l' [DET]
origine [NOUN]
. [PUNCT]


**3b.** Veuillez afficher les tags corrects de `premiere_phrase_test`, puis comparez-les visuellement les tags trouvés automatiquement au (3a).  Quelles différences trouvez-vous ?

In [111]:
print("Tags réels - Tags trouvés")
for token_test, token_doc in zip(premiere_phrase_test, premiere_phrase_test_doc):
    print(f"{token_test.text} [{token_test.pos_}] - {token_doc.text} [{token_doc.pos_}]")
# On trouve aucune différence ! C'est très bien 

Tags réels - Tags trouvés
Je [PRON] - Je [PRON]
sens [VERB] - sens [VERB]
qu' [SCONJ] - qu' [SCONJ]
entre [ADP] - entre [ADP]
ça [PRON] - ça [PRON]
et [CCONJ] - et [CCONJ]
les [DET] - les [DET]
films [NOUN] - films [NOUN]
de [ADP] - de [ADP]
médecins [NOUN] - médecins [NOUN]
et [CCONJ] - et [CCONJ]
scientifiques [NOUN] - scientifiques [NOUN]
fous [ADJ] - fous [PRON]
que [PRON] - que [PRON]
nous [PRON] - nous [PRON]
avons [AUX] - avons [AUX]
déjà [ADV] - déjà [ADV]
vus [VERB] - vus [VERB]
, [PUNCT] - , [PUNCT]
nous [PRON] - nous [PRON]
pourrions [VERB] - pourrions [VERB]
emprunter [VERB] - emprunter [VERB]
un [DET] - un [DET]
autre [ADJ] - autre [ADJ]
chemin [NOUN] - chemin [NOUN]
pour [ADP] - pour [ADP]
l' [DET] - l' [DET]
origine [NOUN] - origine [NOUN]
. [PUNCT] - . [PUNCT]


In [112]:
from spacy.scorer import Scorer
from spacy.training import Example

In [113]:
scorer = Scorer()

**3c.** Au lieu de compter manuellement combien de tags sont différents entre la référence et le résultat de la pipeline `nlp`, vous allez utiliser la classe `Scorer` de spaCy.  Une instance de cette classe permet de calculer les scores d'une liste d'objets de type `Exemple`, en fonction des annotations disponibles dans les objets.  Un objet de type `Exemple` contient deux objets de type `Doc`, l'un avec les annotations correctes et l'autre avec les annotations produites par une pipeline.  La [documentation de la méthode](https://spacy.io/api/scorer#score) `Scorer.score(..)` vous sera utile. 

* Veuillez calculer la justesse (*accuracy*) du *POS tagging* de `premiere_phrase_test`. 
* Veuillez justifier la valeur du score obtenu en utilisant votre réponse du (3b).

In [114]:
example = Example.from_dict(premiere_phrase_test_doc, {"pos": [t.pos_ for t in premiere_phrase_test]})
scores = scorer.score([example])
print(f"POS accuracy: {scores['pos_acc']}")


POS accuracy: 0.9655172413793104


**3d.** Veuillez calculer la précision du *POS tagging* de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se compare le score obtenu avec celui mentionné [dans la documentation](https://spacy.io/models/fr#fr_core_news_sm) du modèle `fr_core_news_sm` ?

In [119]:
# Veuillez écrire votre code ici, suivi de votre réponse à la question.
examples = []
for doc in test_data.get_docs(nlp.vocab):
    pred_doc = nlp(doc.copy())
    example = Example.from_dict(pred_doc, {"pos": [t.pos_ for t in doc]})
    examples.append(example)
    
scores = scorer.score(examples)
print(f"POS accuracy on test data: {scores['pos_acc']:.2%}")



POS accuracy on test data: 91.77%


In [82]:
# On obtient une précision de 0.918, dans la documentation on trouve 0.96 donc c'est bien moins bon ? Après je ne sais pas trop pourquoi

## 4. Entraîner puis évaluer un nouveau POS tagger français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent.  Les [instructions d'entraînement](https://spacy.io/usage/training#quickstart) de spaCy vous montrent comment entraîner une pipeline avec un POS tagger.

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous voulez seulement un POS tagger dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [83]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Enfin, veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques, pour estimer le temps nécessaire et observer les messages affichés.  Puis augmentez progressivement le nombre d'époques.  Quel est le critère qui vous permet de décider que vous avez un nombre suffisant d'époques ?  Dans quel dossier se trouve le meilleur modèle ?

In [84]:
import subprocess

command = [
    "python", "-m", "spacy", "train", "config.cfg",
    "--output", "./myPOStagger1",
    "--paths.train", "./spacy_data/fr-ud-train.spacy",
    "--paths.dev", "./spacy_data/fr-ud-dev.spacy",
    "--verbose",
    "--gpu-id", "0"
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

process.stdout.close()
process.wait()

ℹ Saving to output directory: myPOStagger1
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE 
---  ------  ------------  -----------  -------  ------
  0       0          0.00       211.77    38.03    0.38
  0     200        315.58     10366.11    90.21    0.90
  0     400        285.29      4504.96    91.68    0.92
  0     600        225.79      3457.26    92.04    0.92
  0     800        217.70      3440.73    92.56    0.93
  0    1000        184.30      2961.38    92.60    0.93
  0    1200        183.23      2939.86    92.87    0.93
  0    1400        171.10      2732.89    93.00    0.93
  1    1600        146.14      2197.64    93.09    0.93
  1    1800        137.23      2008.14    93.20    0.93
  1    2000        153.51  

0

In [85]:
# Veuillez indiquer ici le nombre d'époques final et la réponse à la question.
# J'ai laissé le nombre d'epoch à 0 pour que ça se fasse automatiquement et on trouve que le nombre d'époque optimal est de 4. J'ai essayé de mettre à 5 mais on s'arrête tout de même à 4 ?
# Le nombre d'essais optimal est déterminé au moment ou la variable loss tagger arrête de diminuer. On voit qu'on est dans minimum local au bout du 4eme epoch

# Le meilleur model se trouve dans ./myPOStagger1/model-best 

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher la *POS tagging accuracy* sur le corpus de test.  Le composant de la pipeline étant un *POS tagger*, vous devrez évaluer la propriété *tag_acc*. 

In [117]:
# Veuillez écrire votre code ici.
nlp2 = spacy.load("./myPOStagger1/model-best")

score_pos_docs_best = scorer.score([
    Example(nlp2(doc.text), doc) for doc in test_data.get_docs(nlp.vocab)
])

print(f"POS tagger accuracy on test data: {score_pos_docs_best.get('tag_acc'):.2%}")


POS tagger accuracy on test data: 93.32%


## 5. Entraîner puis évaluer un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger appelé *Averaged Perceptron* fourni par NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus, importées cette fois-ci avec NLTK.  Pour une introduction au POS tagging avec NLTK, voir le [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html).

Remarques :
* pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK ;
* pour appliquer un tagger existant, on écrit `nltk.pos_tag(sentence)` où `sentence` est une liste de tokens et on obtient des paires (token, TAG) ;
* l'implémentation de *Averaged Perceptron* a été faite par [Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python), la société qui a créé spaCy.

**5a.** Veuillez charger les données d'entraînement et celles de test grâce à la classe `ConllCorpusReader` de NLTK.  [La documentation de cette classe](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader) vous montrera comment indiquer les colonnes qui contiennent les tokens ('words') et les tags corrects ('pos').  Une fois les données chargées dans une variable, vous pouvez accéder aux phrases et aux tags avec la méthode `.tagged_sents()`.

In [87]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [88]:
# Veuillez écrire votre code ici.
columntypes = ['ignore', 'words', 'ignore', 'pos']

train_data = ConllCorpusReader('./input_data', 'fr-ud-train.conllu', columntypes,separator="\t")
test_data = ConllCorpusReader('./input_data', 'fr-ud-test.conllu', columntypes,separator="\t")

train_sents = train_data.tagged_sents()
test_sents = test_data.tagged_sents()

print(train_sents[0])

[('Les', 'DET'), ('commotions', 'NOUN'), ('cérébrales', 'ADJ'), ('sont', 'AUX'), ('devenu', 'VERB'), ('si', 'ADV'), ('courantes', 'ADJ'), ('dans', 'ADP'), ('ce', 'DET'), ('sport', 'NOUN'), ("qu'", 'SCONJ'), ('on', 'PRON'), ('les', 'PRON'), ('considére', 'VERB'), ('presque', 'ADV'), ('comme', 'ADP'), ('la', 'DET'), ('routine', 'NOUN'), ('.', 'PUNCT')]


**5b.** Pour entraîner un POS tagger du type Averaged Perceptron, vous utiliserez le sous-module `nltk.tag.perceptron` du [module NLTK contenant les taggers](http://www.nltk.org/api/nltk.tag.html).  Les fonctions d'entraînement et de test sont documentées dans ce module.  Après l'entraînement, le réseau de neurones est enregistré dans un fichier `.pickle`, qui est écrasé à chaque entraînement si vous n'en faites pas une copie.  On peut également lire un fichier `.pickle` dans un tagger.

Veuillez écrire le code pour entraîner le POS tagger sur les données d'entraînement.  Comme au (4), pensez augmenter graduellement le nombre d'époques (appelées 'itérations' dans NLTK).

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier enregistré ?

In [89]:
import os
# import nltk
# nltk.download('averaged_perceptron_tagger') # à exécuter la première fois
from perceptron_patched import PerceptronTagger

In [90]:
ptagger = PerceptronTagger(load=False)

In [91]:
# Veuillez écrire votre code ici.
import time 
import os

os.makedirs("./ptaggerOutput", exist_ok=True)

nbIteration = [2,3,5,10,20]

for it in nbIteration:
    start_time = time.time()
    model_path = f"./ptaggerOutput/ptaggermodel-{it}.pickle"
    
    ptagger.train(train_sents, save_loc=model_path, nr_iter=it) 

    training_time = time.time() - start_time
    print(f"Temps d'entraînement pour {it} epochs: {training_time:.2f} secondes")
    
    file_size_bytes = os.path.getsize(model_path + "_averaged_perceptron_tagger.xxx.classes.json")
    file_size_bytes += os.path.getsize(model_path + "_averaged_perceptron_tagger.xxx.tagdict.json")
    file_size_bytes += os.path.getsize(model_path + "_averaged_perceptron_tagger.xxx.weights.json")
    file_size_mb = file_size_bytes / (1024 * 1024) 
    print(f"Taille des fichiers pour {it} epochs: {file_size_mb:.2f} MB")


Temps d'entraînement pour 2 epochs: 9.67 secondes
Taille des fichiers pour 2 epochs: 4.48 MB
Temps d'entraînement pour 3 epochs: 12.41 secondes
Taille des fichiers pour 3 epochs: 5.69 MB
Temps d'entraînement pour 5 epochs: 19.71 secondes
Taille des fichiers pour 5 epochs: 6.52 MB
Temps d'entraînement pour 10 epochs: 38.37 secondes
Taille des fichiers pour 10 epochs: 7.16 MB
Temps d'entraînement pour 20 epochs: 80.25 secondes
Taille des fichiers pour 20 epochs: 7.60 MB


In [ ]:
# Veuillez écrire ici vos réponses aux questions (temps d'entraînement et taille du modèle).
# J'ai additionné tous les fichiers générés pour le modèle et les informations sont dans l'output ci-dessus

**5c.** Veuillez évaluer le tagger sur les données de test et afficher le taux de correction.

In [93]:
# Veuillez écrire votre code ici.
# Load each trained model and evaluate it
for it in nbIteration:
    model_path = f"./ptaggerOutput/ptaggermodel-{it}.pickle"
    
    # Load the model
    tagger = PerceptronTagger(load=False)
    tagger.load_from_json(model_path)
    
    # Calculate accuracy on test data
    accuracy = tagger.accuracy(test_sents)
    
    print(f"Modèle pour {it} epochs:")
    print(f"Precision du POS sur les données de test : {accuracy:.2%}")

Modèle pour 2 epochs:
Precision du POS sur les données de test : 95.67%
Modèle pour 3 epochs:
Precision du POS sur les données de test : 95.75%
Modèle pour 5 epochs:
Precision du POS sur les données de test : 95.92%
Modèle pour 10 epochs:
Precision du POS sur les données de test : 95.95%
Modèle pour 20 epochs:
Precision du POS sur les données de test : 96.06%


## 6. Conclusion

Veuillez remplir le tableau suivant avec les scores obtenus et discuter brièvement comment se comparent les trois POS taggers sur ces données de test.

| spaCy (partie 3) | spaCy (partie 4) | NLTK (partie 5) | 
|------------------|------------------|-----------------|
| tagger fourni    | tagger entraîné  | tagger entraîné |
| 91.77%           |  93.32%          | 96.06%          |


**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.